# Portfolio analysis

In [ ]:
import yfinance as yf
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from scipy import stats
from datetime import datetime, timedelta
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings; warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid'); sns.set_palette("husl")

## 1. Portfolio Setup

**Choose your input method:**
- Use section 1A above for transaction-based input (tracks buy/sell history)
- OR use the manual configuration below for simple percentage-based allocation

## 1A. Easy Portfolio Input (Optional)

**Use this interactive method to input your transactions instead of manual configuration below.**

Enter your buy/sell transactions and the system will automatically calculate your current positions and weights.

In [ ]:
# Easy Portfolio Input System
# Enter your transactions below:

transactions = [
    # Format: {'ticker': 'SYMBOL', 'action': 'buy/sell', 'shares': number, 'date': 'YYYY-MM-DD', 'price': price}
    # Examples:
    {'ticker': 'AAPL', 'action': 'buy', 'shares': 10, 'date': '2023-01-15', 'price': 150.00},
    {'ticker': 'AAPL', 'action': 'buy', 'shares': 5, 'date': '2023-06-20', 'price': 180.00},
    {'ticker': 'MSFT', 'action': 'buy', 'shares': 8, 'date': '2023-03-10', 'price': 280.00},
    {'ticker': 'GOOGL', 'action': 'buy', 'shares': 15, 'date': '2023-02-05', 'price': 95.00},
    {'ticker': 'TSLA', 'action': 'buy', 'shares': 12, 'date': '2023-04-12', 'price': 180.00},
    {'ticker': 'TSLA', 'action': 'sell', 'shares': 5, 'date': '2024-01-15', 'price': 240.00},
]

# Calculate current positions
positions = {}
for t in transactions:
    ticker = t['ticker']
    if ticker not in positions:
        positions[ticker] = {'shares': 0, 'total_cost': 0, 'transactions': []}
    
    if t['action'].lower() == 'buy':
        positions[ticker]['shares'] += t['shares']
        positions[ticker]['total_cost'] += t['shares'] * t['price']
    elif t['action'].lower() == 'sell':
        positions[ticker]['shares'] -= t['shares']
        avg_cost = positions[ticker]['total_cost'] / (positions[ticker]['shares'] + t['shares'])
        positions[ticker]['total_cost'] -= t['shares'] * avg_cost
    
    positions[ticker]['transactions'].append(t)

# Get current prices
portfolio_stocks = [ticker for ticker, pos in positions.items() if pos['shares'] > 0]
current_prices = yf.download(portfolio_stocks, period='1d', progress=False)['Close']
if len(portfolio_stocks) == 1:
    current_prices = pd.Series({portfolio_stocks[0]: current_prices.iloc[-1]})
else:
    current_prices = current_prices.iloc[-1]

# Calculate current values and weights
print("="*80)
print("CURRENT PORTFOLIO POSITIONS")
print("="*80)
total_value = 0
position_values = {}

for ticker in portfolio_stocks:
    shares = positions[ticker]['shares']
    current_price = current_prices[ticker]
    current_value = shares * current_price
    total_cost = positions[ticker]['total_cost']
    profit_loss = current_value - total_cost
    profit_loss_pct = (profit_loss / total_cost) * 100 if total_cost > 0 else 0
    
    position_values[ticker] = current_value
    total_value += current_value
    
    print(f"\n{ticker}:")
    print(f"  Shares: {shares:,.2f}")
    print(f"  Avg Cost: ${total_cost/shares:,.2f}")
    print(f"  Current Price: ${current_price:,.2f}")
    print(f"  Current Value: ${current_value:,.2f}")
    print(f"  P/L: ${profit_loss:,.2f} ({profit_loss_pct:+.2f}%)")

print("\n" + "="*80)
print(f"TOTAL PORTFOLIO VALUE: ${total_value:,.2f}")
print("="*80)

# Calculate weights
portfolio_weights = [position_values[ticker] / total_value for ticker in portfolio_stocks]

print("\nAUTO-CALCULATED WEIGHTS:")
for ticker, weight in zip(portfolio_stocks, portfolio_weights):
    print(f"  {ticker}: {weight*100:.2f}%")

print("\n" + "="*80)
print("✓ Portfolio configuration complete! Continue with the analysis below.")
print("="*80)

---
**OR use the manual method below if you prefer:**

In [ ]:
# Manual Portfolio Configuration (skip if you used section 1A above)
portfolio_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
portfolio_weights = [0.25, 0.25, 0.20, 0.15, 0.15]
benchmark, risk_free_rate = '^GSPC', 0.04
end_date, start_date = datetime.now(), datetime.now() - timedelta(days=365*3)
print(f"Portfolio: {portfolio_stocks}\nWeights: {portfolio_weights}\nBenchmark: {benchmark}\nPeriod: {start_date.date()} to {end_date.date()}")

## 2. Data Collection

Download historical price data for portfolio stocks and benchmark.

In [ ]:
# Download data
stock_data = yf.download(portfolio_stocks, start=start_date, end=end_date, progress=False).xs('Close', level=0, axis=1) if len(portfolio_stocks) > 1 else yf.download(portfolio_stocks, start=start_date, end=end_date, progress=False)[['Close']].rename(columns={'Close': portfolio_stocks[0]})
benchmark_data = yf.download(benchmark, start=start_date, end=end_date, progress=False)['Close'].squeeze()
print("Stock Data:\n", stock_data.head(), "\n\nBenchmark Data:\n", benchmark_data.head())

## 3. Calculate Returns

Calculate daily returns for stocks and benchmark.

In [ ]:
# Calculate returns
stock_returns, benchmark_returns = stock_data.pct_change().dropna(), benchmark_data.pct_change().dropna()
portfolio_returns = (stock_returns * portfolio_weights).sum(axis=1)
cumulative_portfolio_returns, cumulative_benchmark_returns = (1 + portfolio_returns).cumprod(), (1 + benchmark_returns).cumprod()
print("Stock Returns:\n", stock_returns.head(), "\n\nPortfolio Returns:\n", portfolio_returns.head())

In [ ]:
# Visualize cumulative returns
plt.figure(figsize=(14, 6))
plt.plot(cumulative_portfolio_returns, label='Portfolio', linewidth=2)
plt.plot(cumulative_benchmark_returns, label='Benchmark (S&P 500)', linewidth=2)
plt.title('Cumulative Returns: Portfolio vs Benchmark', fontsize=16, fontweight='bold')
plt.xlabel('Date'); plt.ylabel('Cumulative Return'); plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()
print(f"\nTotal Portfolio Return: {(cumulative_portfolio_returns.iloc[-1] - 1) * 100:.2f}%\nTotal Benchmark Return: {(cumulative_benchmark_returns.iloc[-1] - 1) * 100:.2f}%")

## 4. Beta Calculation

Beta measures the portfolio's volatility relative to the market.

In [ ]:
# Calculate Beta
beta = np.cov(portfolio_returns, benchmark_returns)[0][1] / np.var(benchmark_returns)
print(f"Portfolio Beta: {beta:.4f}\nInterpretation: Portfolio is {abs((beta - 1) * 100):.2f}% {'more' if beta > 1 else 'less'} volatile than the market")
plt.figure(figsize=(10, 6)); plt.scatter(benchmark_returns, portfolio_returns, alpha=0.5)
z = np.polyfit(benchmark_returns, portfolio_returns, 1); plt.plot(benchmark_returns, np.poly1d(z)(benchmark_returns), "r-", linewidth=2, label=f'Beta = {beta:.4f}')
plt.xlabel('Benchmark Returns'); plt.ylabel('Portfolio Returns'); plt.title('Portfolio Beta (Systematic Risk)', fontsize=16, fontweight='bold')
plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()

## 5. CAPM (Capital Asset Pricing Model)

CAPM calculates the expected return based on systematic risk (Beta).

In [ ]:
# CAPM Analysis
annual_benchmark_return, annual_portfolio_return = benchmark_returns.mean() * 252, portfolio_returns.mean() * 252
expected_return_capm = risk_free_rate + beta * (annual_benchmark_return - risk_free_rate)
print(f"CAPM Analysis:\n  Risk-Free Rate: {risk_free_rate * 100:.2f}%\n  Market Return: {annual_benchmark_return * 100:.2f}%\n  Market Risk Premium: {(annual_benchmark_return - risk_free_rate) * 100:.2f}%\n  Portfolio Beta: {beta:.4f}\n  Expected Return (CAPM): {expected_return_capm * 100:.2f}%\n  Actual Portfolio Return: {annual_portfolio_return * 100:.2f}%")

## 6. Alpha Calculation

Alpha measures excess return above what CAPM predicts (risk-adjusted outperformance).

In [ ]:
# Calculate Alpha
alpha = annual_portfolio_return - expected_return_capm
print(f"Portfolio Alpha: {alpha * 100:.2f}%\n{'✓ Positive Alpha: Outperformed by ' + f'{alpha * 100:.2f}%' if alpha > 0 else '✗ Negative Alpha: Underperformed by ' + f'{abs(alpha) * 100:.2f}%'}")

## 7. Sharpe Ratio

Sharpe Ratio measures risk-adjusted return (return per unit of total risk).

In [ ]:
# Sharpe Ratio
portfolio_std, benchmark_std = portfolio_returns.std() * np.sqrt(252), benchmark_returns.std() * np.sqrt(252)
sharpe_ratio, benchmark_sharpe = (annual_portfolio_return - risk_free_rate) / portfolio_std, (annual_benchmark_return - risk_free_rate) / benchmark_std
print(f"Sharpe Ratio Analysis:\n  Portfolio: {sharpe_ratio:.4f}\n  Benchmark: {benchmark_sharpe:.4f}\n  Portfolio Volatility: {portfolio_std * 100:.2f}%\n  Benchmark Volatility: {benchmark_std * 100:.2f}%\n  {'✓ Portfolio has better risk-adjusted returns' if sharpe_ratio > benchmark_sharpe else '✗ Benchmark has better risk-adjusted returns'}")

## 8. Treynor Ratio

Treynor Ratio measures risk-adjusted return per unit of systematic risk (Beta).

In [ ]:
# Treynor Ratio
treynor_ratio, benchmark_treynor = (annual_portfolio_return - risk_free_rate) / beta, (annual_benchmark_return - risk_free_rate) / 1.0
print(f"Treynor Ratio:\n  Portfolio: {treynor_ratio:.4f}\n  Benchmark: {benchmark_treynor:.4f}\n  {'✓ Portfolio has better return per unit of systematic risk' if treynor_ratio > benchmark_treynor else '✗ Benchmark is better'}")

## 9. Omega Ratio

Omega Ratio considers the probability of achieving returns above a threshold.

In [ ]:
# Omega Ratio
calc_omega = lambda r, t: (r[r > t] - t).sum() / (t - r[r < t]).sum() if (t - r[r < t]).sum() > 0 else np.inf
daily_rf = risk_free_rate / 252
omega_ratio_portfolio, omega_ratio_benchmark = calc_omega(portfolio_returns, daily_rf), calc_omega(benchmark_returns, daily_rf)
print(f"Omega Ratio:\n  Portfolio: {omega_ratio_portfolio:.4f}\n  Benchmark: {omega_ratio_benchmark:.4f}\n  {'✓ Portfolio has better gain/loss ratio' if omega_ratio_portfolio > omega_ratio_benchmark else '✗ Benchmark is better'}")

## 10. Value at Risk (VaR)

VaR estimates the maximum potential loss over a specific time period at a given confidence level.

In [ ]:
# Value at Risk
confidence_levels = [0.90, 0.95, 0.99]
print("Value at Risk (VaR):\n" + "="*70)
print("\n1. Historical VaR:")
for c in confidence_levels: print(f"   {c*100:.0f}% Confidence: {np.percentile(portfolio_returns, (1-c)*100)*100:.4f}% daily loss (${10000*abs(np.percentile(portfolio_returns, (1-c)*100)):.2f} on $10k)")
print("\n2. Parametric VaR:")
for c in confidence_levels: print(f"   {c*100:.0f}% Confidence: {(portfolio_returns.mean() + stats.norm.ppf(1-c)*portfolio_returns.std())*100:.4f}% daily loss")
print("\n3. Conditional VaR (CVaR):")
for c in confidence_levels: print(f"   {c*100:.0f}% Confidence: {portfolio_returns[portfolio_returns <= np.percentile(portfolio_returns, (1-c)*100)].mean()*100:.4f}% daily loss")

In [ ]:
# Visualize VaR
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
var_95 = np.percentile(portfolio_returns, 5)
ax1.hist(portfolio_returns, bins=50, alpha=0.7, edgecolor='black')
ax1.axvline(var_95, color='r', linestyle='--', linewidth=2, label=f'95% VaR: {var_95*100:.2f}%')
ax1.axvline(portfolio_returns.mean(), color='g', linestyle='--', linewidth=2, label=f'Mean: {portfolio_returns.mean()*100:.2f}%')
ax1.set_xlabel('Daily Returns'); ax1.set_ylabel('Frequency'); ax1.set_title('Distribution with VaR', fontsize=14, fontweight='bold'); ax1.legend(); ax1.grid(True, alpha=0.3)
stats.probplot(portfolio_returns, dist="norm", plot=ax2); ax2.set_title('Q-Q Plot (Normality Check)', fontsize=14, fontweight='bold'); ax2.grid(True, alpha=0.3)
plt.tight_layout(); plt.show()

## 11. Principal Component Analysis (PCA)

PCA identifies the main sources of portfolio variance and correlation structure.

In [ ]:
# PCA Analysis
pca = PCA(); pca_components = pca.fit_transform(StandardScaler().fit_transform(stock_returns))
explained_variance, cumulative_variance = pca.explained_variance_ratio_, np.cumsum(pca.explained_variance_ratio_)
loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(len(portfolio_stocks))], index=portfolio_stocks)
print("PCA:\n" + "="*70)
for i, (v, cv) in enumerate(zip(explained_variance, cumulative_variance), 1): print(f"PC{i}: {v*100:.2f}% variance | Cumulative: {cv*100:.2f}%")
print("\n" + "="*70 + f"\nLoadings:\n{loadings.round(3)}")

In [ ]:
# Visualize PCA
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
axes[0].bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, label='Individual')
axes[0].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'ro-', linewidth=2, label='Cumulative')
axes[0].set_xlabel('PC'); axes[0].set_ylabel('Variance'); axes[0].set_title('Scree Plot', fontweight='bold'); axes[0].legend(); axes[0].grid(True, alpha=0.3)
sns.heatmap(loadings.iloc[:, :3], annot=True, cmap='coolwarm', center=0, ax=axes[1], fmt='.2f'); axes[1].set_title('Component Loadings', fontweight='bold')
sns.heatmap(stock_returns.corr(), annot=True, cmap='coolwarm', center=0, ax=axes[2], fmt='.2f'); axes[2].set_title('Correlation Matrix', fontweight='bold')
plt.tight_layout(); plt.show()
print(f"First PC: {explained_variance[0]*100:.2f}%, First 2 PCs: {cumulative_variance[1]*100:.2f}%")

## 12. Monte Carlo Simulation

Simulate thousands of possible future portfolio scenarios to estimate risk and return distributions.

In [ ]:
# Monte Carlo Setup
num_simulations, num_days, initial_investment = 10000, 252, 10000
mean_returns, cov_matrix = stock_returns.mean(), stock_returns.cov()
print(f"Running {num_simulations:,} Monte Carlo simulations for {num_days} trading days...")

In [ ]:
# Run Monte Carlo
np.random.seed(42)
simulation_results = np.array([initial_investment * (1 + np.dot(np.random.multivariate_normal(mean_returns, cov_matrix, num_days), portfolio_weights)).cumprod() for _ in range(num_simulations)]).T
final_values = simulation_results[-1, :]
mean_final_value, median_final_value = final_values.mean(), np.median(final_values)
print(f"✓ Complete!\n{'='*70}\nResults (1 Year):\n{'='*70}\nMean: ${mean_final_value:,.2f} | Median: ${median_final_value:,.2f}\n5th: ${np.percentile(final_values, 5):,.2f} | 95th: ${np.percentile(final_values, 95):,.2f}\nProb of Loss: {(final_values < initial_investment).sum() / num_simulations * 100:.2f}% | Expected Return: {((mean_final_value - initial_investment) / initial_investment) * 100:.2f}%")

In [ ]:
# Visualize Monte Carlo
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes[0, 0].plot(simulation_results, alpha=0.01, color='blue'); axes[0, 0].axhline(y=initial_investment, color='r', linestyle='--', linewidth=2)
axes[0, 0].plot(simulation_results.mean(axis=1), color='orange', linewidth=3); axes[0, 0].set_title(f'{num_simulations:,} Simulations', fontweight='bold')
axes[0, 1].hist(final_values, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
for val, col, lbl in [(initial_investment, 'r', 'Initial'), (mean_final_value, 'orange', f'Mean: ${mean_final_value:,.0f}')]: axes[0, 1].axvline(val, color=col, linestyle='--', linewidth=2, label=lbl)
axes[0, 1].set_title('Final Values Distribution', fontweight='bold'); axes[0, 1].legend()
for p in [5, 25, 50, 75, 95]: axes[1, 0].plot(np.percentile(simulation_results, p, axis=1), label=f'{p}th %ile', linewidth=2)
axes[1, 0].axhline(y=initial_investment, color='r', linestyle='--', linewidth=2); axes[1, 0].set_title('Percentile Bands', fontweight='bold'); axes[1, 0].legend()
final_returns = ((final_values - initial_investment) / initial_investment) * 100
axes[1, 1].hist(final_returns, bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
axes[1, 1].axvline(0, color='r', linestyle='--', linewidth=2); axes[1, 1].set_title('Returns Distribution', fontweight='bold')
plt.tight_layout(); plt.show()

## 13. Summary Dashboard

Comprehensive overview of all portfolio metrics.

In [ ]:
# Summary Dashboard
summary_df = pd.DataFrame({
    'Metric': ['Annual Return', 'Volatility', 'Sharpe', 'Treynor', 'Alpha', 'Beta', 'Omega', 'VaR 95%', 'CVaR 95%', 'Max DD', 'Total Return'],
    'Portfolio': [f"{annual_portfolio_return*100:.2f}%", f"{portfolio_std*100:.2f}%", f"{sharpe_ratio:.4f}", f"{treynor_ratio:.4f}", f"{alpha*100:.2f}%", f"{beta:.4f}", 
                  f"{omega_ratio_portfolio:.4f}", f"{np.percentile(portfolio_returns, 5)*100:.4f}%", f"{portfolio_returns[portfolio_returns <= np.percentile(portfolio_returns, 5)].mean()*100:.4f}%",
                  f"{((cumulative_portfolio_returns / cumulative_portfolio_returns.cummax() - 1).min() * 100):.2f}%", f"{(cumulative_portfolio_returns.iloc[-1] - 1)*100:.2f}%"],
    'Benchmark': [f"{annual_benchmark_return*100:.2f}%", f"{benchmark_std*100:.2f}%", f"{benchmark_sharpe:.4f}", f"{benchmark_treynor:.4f}", "0.00%", "1.0000",
                  f"{omega_ratio_benchmark:.4f}", f"{np.percentile(benchmark_returns, 5)*100:.4f}%", f"{benchmark_returns[benchmark_returns <= np.percentile(benchmark_returns, 5)].mean()*100:.4f}%",
                  f"{((cumulative_benchmark_returns / cumulative_benchmark_returns.cummax() - 1).min() * 100):.2f}%", f"{(cumulative_benchmark_returns.iloc[-1] - 1)*100:.2f}%"]
})
print("\n" + "="*80 + "\nPORTFOLIO ANALYSIS SUMMARY\n" + "="*80); print(summary_df.to_string(index=False))
print("\n" + "="*80 + "\nMONTE CARLO (1-YEAR)\n" + "="*80)
print(f"Mean: ${mean_final_value:,.2f} | Median: ${median_final_value:,.2f}\n5th: ${np.percentile(final_values, 5):,.2f} | 95th: ${np.percentile(final_values, 95):,.2f}\nLoss Prob: {(final_values < initial_investment).sum() / num_simulations * 100:.2f}% | Exp Return: {((mean_final_value - initial_investment) / initial_investment) * 100:.2f}%\n" + "="*80)

In [ ]:
# Visual Dashboard
fig = plt.figure(figsize=(16, 10)); gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
ax1 = fig.add_subplot(gs[0, :2]); ax1.plot(cumulative_portfolio_returns, label='Portfolio', linewidth=2.5); ax1.plot(cumulative_benchmark_returns, label='Benchmark', linewidth=2.5)
ax1.fill_between(cumulative_portfolio_returns.index, cumulative_portfolio_returns, cumulative_benchmark_returns, where=(cumulative_portfolio_returns > cumulative_benchmark_returns), alpha=0.3, color='green')
ax1.set_title('Cumulative Returns', fontweight='bold'); ax1.legend(); ax1.grid(True, alpha=0.3)
ax2 = fig.add_subplot(gs[0, 2]); x = np.arange(3); width = 0.35
ax2.bar(x - width/2, [sharpe_ratio, treynor_ratio, omega_ratio_portfolio], width, label='Portfolio', alpha=0.8)
ax2.bar(x + width/2, [benchmark_sharpe, benchmark_treynor, omega_ratio_benchmark], width, label='Benchmark', alpha=0.8)
ax2.set_xticks(x); ax2.set_xticklabels(['Sharpe', 'Treynor', 'Omega']); ax2.set_title('Risk-Adjusted Returns', fontweight='bold'); ax2.legend()
ax3 = fig.add_subplot(gs[1, 0]); ax3.pie(portfolio_weights, labels=portfolio_stocks, autopct='%1.1f%%', startangle=90); ax3.set_title('Allocation', fontweight='bold')
ax4 = fig.add_subplot(gs[1, 1]); ax4.axis('off')
risk_table = [['Metric', 'Value'], ['Beta', f'{beta:.4f}'], ['Alpha', f'{alpha*100:.2f}%'], ['VaR 95%', f'{np.percentile(portfolio_returns, 5)*100:.2f}%']]
table = ax4.table(cellText=risk_table, cellLoc='left', loc='center'); table.set_fontsize(10); table.scale(1, 2); ax4.set_title('Key Metrics', fontweight='bold', pad=20)
ax5 = fig.add_subplot(gs[1, 2]); ax5.hist(portfolio_returns, bins=40, alpha=0.7, color='skyblue', edgecolor='black'); ax5.axvline(portfolio_returns.mean(), color='red', linestyle='--', linewidth=2)
ax5.set_title('Returns Distribution', fontweight='bold')
ax6 = fig.add_subplot(gs[2, :])
for p in [5, 50, 95]: ax6.plot(np.percentile(simulation_results, p, axis=1), label=f'{p}th %ile', linewidth=2)
ax6.axhline(y=initial_investment, color='r', linestyle='--', linewidth=2); ax6.fill_between(range(num_days), np.percentile(simulation_results, 5, axis=1), np.percentile(simulation_results, 95, axis=1), alpha=0.2)
ax6.set_title('Monte Carlo: 1-Year Projection', fontweight='bold'); ax6.legend()
plt.suptitle('PORTFOLIO ANALYSIS DASHBOARD', fontsize=18, fontweight='bold', y=0.995); plt.show()

## 14. Key Insights & Recommendations

Based on the analysis above, here are key insights about your portfolio.

In [ ]:
# Key Insights
var_95 = np.percentile(portfolio_returns, 5); first_pc_variance = explained_variance[0] * 100
score = sum([sharpe_ratio > benchmark_sharpe, alpha > 0, 0.8 <= beta <= 1.2, first_pc_variance < 70, abs(var_95) <= 0.03])
print("="*80 + "\nKEY INSIGHTS & RECOMMENDATIONS\n" + "="*80)
print(f"\n1. RISK-ADJUSTED: {'✓' if sharpe_ratio > benchmark_sharpe else '⚠'} Sharpe {sharpe_ratio:.3f} vs {benchmark_sharpe:.3f}")
print(f"2. ALPHA: {'✓' if alpha > 0 else '⚠'} {alpha*100:.2f}% {'outperformance' if alpha > 0 else 'underperformance'}")
print(f"3. VOLATILITY: {'⚠' if beta > 1.2 else '✓'} Beta {beta:.3f} - {abs((beta-1)*100):.0f}% {'more' if beta > 1 else 'less'} volatile")
print(f"4. DIVERSIFICATION: {'✓' if first_pc_variance < 70 else '⚠'} First PC explains {first_pc_variance:.1f}%")
print(f"5. DOWNSIDE RISK: {'✓' if abs(var_95) <= 0.03 else '⚠'} VaR 95%: {var_95*100:.2f}% (${abs(var_95)*10000:.2f} on $10k)")
print(f"6. OUTLOOK: Loss prob {(final_values < initial_investment).sum() / num_simulations * 100:.1f}% | Expected return {((mean_final_value - initial_investment) / initial_investment) * 100:.2f}%")
print("\n" + "="*80 + "\nOVERALL ASSESSMENT\n" + "="*80)
print(['⚠ NEEDS IMPROVEMENT', '⚠ MODERATE PORTFOLIO', '✓ GOOD PORTFOLIO', '✓ GOOD PORTFOLIO', '✓ STRONG PORTFOLIO', '✓ STRONG PORTFOLIO'][score])
print(f"Portfolio Score: {score}/5\n" + "="*80)